In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from datasetUtils import load_dataset
from TranslationModel import TranslationModel
import rouge
import tensorflow as tf
import numpy as np
from utils  import preprocess_sentence
import metrics

In [3]:
# fix tensorflow out of memory problem
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

NameError: name 'tf' is not defined

In [2]:
print("Loading the dataset  ...")
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset("./API+JAVADOC+CODE.json")
# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor,
                                                                                                  target_tensor,
                                                                                                test_size=0.2)


#Parameters
BUFFER_SIZE = len(input_tensor_train)
batch_size = 16
embedding_dim = 512
units = 256
epochs = 20
vocab_inp_size = len(inp_lang.word_index) + 1
vocab_tar_size = len(targ_lang.word_index) + 1
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction='none')


print("Loading the model ...")
model = TranslationModel(vocab_inp_size=vocab_inp_size, vocab_tar_size=vocab_tar_size,
                         units=units,
                         embedding_dim=embedding_dim,
                         optimizer=optimizer,
                         max_length_inp=max_length_inp,
                         max_length_targ=max_length_targ,
                         loss_object=loss_object,
                         batch_size=batch_size
                         )

print("start training ...")
model.train(input_tensor_train=input_tensor_train, target_tensor_train=target_tensor_train,
            target_lang=targ_lang, input_tensor_val=input_tensor_val, target_tensor_val=target_tensor_val)

Loading the dataset  ...
Loading the model ...
start training ...
Initializing from scratch.
Current Epoch  0
Epoch 1 Batch 1 Loss 3.0854
Epoch 1 Batch 101 Loss 2.6801
Epoch 1 Batch 201 Loss 2.1105
Epoch 1 Batch 301 Loss 1.7670
Epoch 1 Batch 401 Loss 1.7573
Epoch 1 Batch 501 Loss 1.7758
Epoch 1 Batch 601 Loss 1.6424
Epoch 1 Batch 701 Loss 1.6066
Epoch 1 Batch 801 Loss 2.1610
Epoch 1 Batch 901 Loss 2.1540
Epoch 1 Batch 1001 Loss 1.7653
Epoch 1 Batch 1101 Loss 1.5276
Epoch 1 Batch 1201 Loss 1.7173
Epoch 1 Batch 1301 Loss 2.2282
Epoch 1 Loss 1.9792
Time taken for 1 epoch 2350.40 sec

Current Epoch  1
Epoch 2 Batch 1 Loss 1.6232
Epoch 2 Batch 101 Loss 1.6819
Epoch 2 Batch 201 Loss 1.2352
Epoch 2 Batch 301 Loss 2.0376
Epoch 2 Batch 401 Loss 1.6086
Epoch 2 Batch 501 Loss 1.3106
Epoch 2 Batch 601 Loss 2.0231
Epoch 2 Batch 701 Loss 1.4395
Epoch 2 Batch 801 Loss 1.1456
Epoch 2 Batch 901 Loss 1.8855
Epoch 2 Batch 1001 Loss 1.7676
Epoch 2 Batch 1101 Loss 1.7884
Epoch 2 Batch 1201 Loss 1.5384
Epoc

In [4]:
def evaluate_single(sentence):
  sentence = preprocess_sentence(sentence)
  inputs = [inp_lang.word_index[i] for i in sentence.split(' ') if i in inp_lang.word_index.keys()]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)
  result = '<start> '
  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = model.encoder(inputs, hidden)
  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = model.decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += targ_lang.index_word[predicted_id] + ' '
    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)
  return result, sentence

def translate(sentence):
  result, sentence = evaluate(sentence)
  print('Input code \n', sentence)    
  print('Predicted javadoc \n', result)

In [4]:
def evaluate(input_tensor_val, target_tensor_val, targ_lang):
    enc_hidden =  tf.zeros((len(input_tensor_val), units))    
    enc_out, enc_hidden = model.encoder(input_tensor_val, enc_hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * len(input_tensor_val), 1)  # (1 * len(input_tensor_val))
    predicted_sequnces = np.zeros((len(input_tensor_val), max_length_targ))
    loss = 0.
    for t in range(max_length_targ):
        # preds (batch_size * voab size)
        preds, dec_hidden, att_weights = model.decoder(dec_input,
                                                      dec_hidden,
                                                      enc_out)

        loss += loss_function(model.loss_object, target_tensor_val[:, t], preds)

        
        # predicted_ids (batch_size * 1)
        predicted_ids = tf.argmax(preds, 1).numpy()
        predicted_sequnces[:, t] = predicted_ids        
        dec_input = tf.expand_dims(predicted_ids, 1)
    
    refs = target_lang.sequences_to_texts(target_tensor_val)
    preds = target_lang.sequences_to_texts(predicted_sequnces)

    with open('./preds.txt') as refs, open('./refs') as preds:
        refs.write([i + \'n' for i in refs])
        preds.write([i + \'n' for i in preds])

    
    blue_score = metrics.calculate_bleu('./preds.txt', './refs')
    rouge_score = metrics.calculate_rouge('./preds.txt', './refs')
    
    loss = loss / int(target_tensor_val.shape[1])        # take the average loss
    
    return loss, blue_score, rouge_score

In [6]:
predicted_sequnces = _evaluate_2(input_tensor_val, target_tensor_val, targ_lang)
sequence_to_texts(predicted_sequnces, input_tensor_val, targ_lang)
#texts = targ_lang.sequences_to_texts(target_tensor_val)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run Identity: Dst tensor is not initialized. [Op:Identity]

In [17]:
sentence = "View getActionItemView(int position) {\n        final RecyclerView.ViewHolder holder = mActionsStylist.getActionsGridView()\n                    .findViewHolderForPosition(position);\n        return holder == null ? null : holder.itemView;\n    }"
#sentence = "void start() { checkStarted(false); mStarted = true; mHeifEncoder.start(); }                 .findViewHolderForPosition(position);\n        return holder == null ? null : holder.itemView;\n    }"
#sentence = "void addAttribute(String name, String type, String value) { names.add(name); types.add(type); values.add(value); }"
#sentence = "final int getVolumeControl() { return mControlType; }"

translate(sentence)

Input code: <start> view getactionitemview int position final recyclerview . viewholder holder mactionsstylist . getactionsgridview . findviewholderforposition position return holder null ? null holder . itemview <end>

Predicted javadoc: <start> int getcolor return drawable <end> 


In [5]:
with open('test.txt', 'w+') as out:
    out.write([i + '\n' for i in ['malek', 'baba']])

TypeError: write() argument must be str, not list